In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
import json
import os
import requests
import io
import zipfile

/tmp/ipykernel_67582/2038222792.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
dfs : dict[str, pd.DataFrame] = { file.split('.')[0] : pd.read_csv(f'../data/core/mb-alloc/{file}', dtype=str) for file in os.listdir('../data/core/mb-alloc')}

In [3]:
dfs = dict(sorted(dfs.items()))

In [4]:
dfs['MB_2021_AUST']['MB_CODE_2021'].is_unique, dfs['SAL_2021_AUST']['MB_CODE_2021'].is_unique, dfs['POA_2021_AUST']['MB_CODE_2021'].is_unique, dfs['SUA_2021_AUST']['SA2_CODE_2021'].is_unique

(True, True, True, True)

In [5]:
dfs['MB_2021_AUST']['SA2_CODE_2021'].unique().size, dfs['SUA_2021_AUST']['SA2_CODE_2021'].unique().size

(2473, 2473)

In [6]:
dfs['MB_2021_AUST']['MB_CODE_2021'].unique().size, dfs['POA_2021_AUST']['MB_CODE_2021'].unique().size

(368286, 368286)

In [7]:
dfs['MB_2021_AUST']['MB_CODE_2021'].unique().size, dfs['POA_2021_AUST']['MB_CODE_2021'].unique().size

(368286, 368286)

In [8]:
dfMB = pd.merge(dfs['MB_2021_AUST'], dfs['SAL_2021_AUST'], on='MB_CODE_2021', how='left', suffixes=('', '_SAL_'))
dfMB = pd.merge(dfMB, dfs['POA_2021_AUST'], on='MB_CODE_2021', how='left', suffixes=('', '_POA_'))
dfMB = pd.merge(dfMB, dfs['LGA_2023_AUST'], on='MB_CODE_2021', how='left', suffixes=('', '_LGA_'))
dfMB = pd.merge(dfMB, dfs['SUA_2021_AUST'], on='SA2_CODE_2021', how='left', suffixes=('', '_SUA_'))
dfMB.drop(columns=[c for c in dfMB.columns if c.endswith('_')], inplace=True)
dfMB['AREA_ALBERS_SQKM'] = dfMB['AREA_ALBERS_SQKM'].astype(float)

In [9]:
dfMB[dfMB['SUA_CODE_2021'].isna()]

,MB_CODE_2021,MB_CATEGORY_2021,CHANGE_FLAG_2021,CHANGE_LABEL_2021,SA1_CODE_2021,SA2_CODE_2021,SA2_NAME_2021,SA3_CODE_2021,SA3_NAME_2021,SA4_CODE_2021,...,AREA_ALBERS_SQKM,ASGS_LOCI_URI_2021,SAL_CODE_2021,SAL_NAME_2021,POA_CODE_2021,POA_NAME_2021,LGA_CODE_2023,LGA_NAME_2023,SUA_CODE_2021,SUA_NAME_2021


In [ ]:
df_suburbs = dfMB.groupby(['SAL_CODE_2021', 'POA_CODE_2021']).aggregate({
    'SAL_NAME_2021' : 'first',
    'POA_NAME_2021' : 'first',
    'GCCSA_CODE_2021' : 'unique',
    'GCCSA_NAME_2021' : 'unique',
    'LGA_CODE_2023' : 'unique',
    'LGA_NAME_2023' : 'unique',
    'SUA_CODE_2021' : 'unique',
    'SUA_NAME_2021' : 'unique',
    'STATE_CODE_2021' : 'first',
    'STATE_NAME_2021' : 'first',
    'AUS_CODE_2021' : 'first',
    'AUS_NAME_2021' : 'first',
    'AREA_ALBERS_SQKM' : 'sum',
    'MB_CATEGORY_2021' : 'unique',
}).reset_index().rename(columns={
    'SAL_CODE_2021' : 'suburb_id',
    'SAL_NAME_2021' : 'suburb_name',
    'POA_CODE_2021' : 'postcode_id',
    'POA_NAME_2021' : 'postcode',
    'GCCSA_CODE_2021' : 'gccsa_id',
    'GCCSA_NAME_2021' : 'capital_area',
    'LGA_CODE_2023' : 'lga_id',
    'LGA_NAME_2023' : 'lga_name',
    'SUA_CODE_2021' : 'sua_id',
    'SUA_NAME_2021' : 'urban_area',
    'STATE_CODE_2021' : 'state_id',
    'STATE_NAME_2021' : 'state_name',
    'AUS_CODE_2021' : 'country_id',
    'AUS_NAME_2021' : 'country_name',
    'AREA_ALBERS_SQKM' : 'area_sqkm',
    'MB_CATEGORY_2021' : 'categories',
})


In [ ]:
sua_ids = dfs['SUA_2021_AUST']['SUA_CODE_2021'].unique()

In [ ]:
dfk_sua_suburbs = {}
for sua_id in sua_ids:
    dfk_sua_suburbs[sua_id] = df_suburbs[df_suburbs['sua_id'].apply(lambda x: sua_id in x)]

In [ ]:
gdf_suburbs : gpd.GeoDataFrame = gpd.GeoDataFrame(pd.concat([gpd.read_file(f'../data/core/suburbs/{file}') for file in os.listdir('../data/core/suburbs')], ignore_index=True))

In [ ]:
len(df_suburbs), len(gdf_suburbs)

In [ ]:
df_suburbs[['suburb_id', 'postcode_id', 'sua_id']].head()

In [ ]:
gdf_suburbs.apply(lambda x: (x['suburb_id'], x['postcode_id']), axis=1).is_unique

In [ ]:
df_suburbs.apply(lambda x: (x['suburb_id'], x['postcode_id']), axis=1).is_unique

In [ ]:
gdf_sua = pd.merge(gdf_suburbs, df_suburbs[['suburb_id', 'postcode_id', 'sua_id']], on=['suburb_id', 'postcode_id'], how='left')

In [ ]:
gdf_sua = gpd.GeoDataFrame(gdf_sua)

In [ ]:
os.makedirs('../data/sua-suburbs', exist_ok=True)

In [ ]:
for sua_id in sua_ids:
    if sua_id == '1000':
        continue # potential file size of sua_id=1000 might be > 100M
    gdf_sua[gdf_sua['sua_id'].apply(lambda x: sua_id in x)].drop(columns=['sua_id']).to_file(f'../data/sua-suburbs/sua-suburbs-{sua_id}.geojson', driver='GeoJSON')
# 1m 30s

In [ ]:
gdf_sua_1000 = gdf_sua[gdf_sua['sua_id'].apply(lambda x: '1000' in x)].drop(columns=['sua_id'])

In [ ]:
gdf_sua_1000[gdf_sua_1000['postcode'] < '2500'].to_file(f'../data/sua-suburbs/sua-suburbs-1000-24.geojson', driver='GeoJSON')

In [ ]:
gdf_sua_1000[gdf_sua_1000['postcode'] >= '2500'].to_file(f'../data/sua-suburbs/sua-suburbs-1000-25.geojson', driver='GeoJSON')

In [15]:
dfs['SUA_2021_AUST'].groupby('SUA_CODE_2021').aggregate({
    'SUA_NAME_2021' : 'first',
    'AUS_CODE_2021' : 'first',
    'AUS_NAME_2021' : 'first'
}).reset_index().rename(columns={
    'SUA_CODE_2021' : 'sua_id',
    'SUA_NAME_2021' : 'sua_name',
    'AUS_CODE_2021' : 'country_id',
    'AUS_NAME_2021' : 'country_name'
}).to_csv('../data/sua-suburbs/sua-info.csv')